# 매출데이터 - 유동인구 데이터 연결
---
## 개요
이전에 생성한 연결 링크 파일을 사용하여, 실제 매출 데이터와 유동인구 데이터간 연결 시행

## 사용 파일
서울교통공사_역별 일별 시간대별 승하차인원_20211231.csv, 서울특별시 열린 데이터 광장, 서율교통공사 제공

서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2021년.csv, 서울특별시 열린 데이터 광장, 제공

sales_district_name_IDcode.csv, 매출데이터의 상권_코드_명과 링크 번호 매칭 데이터

subway_station_name_IDcode.csv, 유동인구데이터의 역명과 링크 번호 매칭 데이터

## 구동 시퀸스
매출 데이터 part
- 매출 데이터에서 타겟 업종(현재:제과점) 별도 추출 = 이하 제과점
- "제과점"과 sales_dis~.csv(매출 링크 데이터)를 연결하여 링크 생성
- 링크가 없는 데이터 행 제거
- 링크를 기준으로하여 같은 역명을 가진 데이터 합산 묶기
- 묶인 데이터에서, 묶여서는 안되는 수치나 문자열들 정리

유동인구 데이터 part
- 유동인구 데이터를 역명을 기준으로하여 요일-시간대 별 데이터로 정리 = 이하 인구데이터
- "인구데이터"와 subway_sta~.csv(유동인구 링크 데이터)를 연결하여 링크 생성

통합
- 링크 생성 및 정리된 "제과점"과 "인구데이터"를 링크 번호를 사용하여 병합

## 주의사항
- 기존에 생성했었던 유동인구 정리데이터를 사용함
    - csvdata 폴더 내 데이터 사용
    - 데이터 생성 방식에 대해선 population_data_simplification.ipynb 참조
- 테스트 및 확인을 위하여 2021_1분기 데이터만 사용하였음

## 기초 import 및 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 매출 및 유동인구 데이터 가져오기
money_data = pd.read_csv("./csvdata/2021_money_add.csv", encoding="EUC-KR", header=0)  # 자치구, 행정동 포함된 매출 데이터
qt1_population_data = pd.read_csv("./csvdata/2021_1qt.csv", encoding='utf-8', header=0)  # 2021_1분기 정리 데이터

In [ ]:
# 링크 데이터 2종 가져오기
linked_df = pd.read_csv("./csvdata/sales_district_name_IDcode.csv")  # 매출 링크 데이터
linked_stt_df = pd.read_csv("./csvdata/subway_station_name_IDcode.csv")  # 유동인구 링크 데이터

## 제과점 데이터 추출 및 정리

In [ ]:
# 제과점 데이터 추출
bakery_data = money_data[money_data['서비스_업종_코드_명'].str.contains("제과")]
bakery_data.reset_index(drop=True, inplace=True)  # contains를 사용하면 index가 연속되지 않기때문에, 초기화

# link라는 컬럼이 있으면 삭제해줌
# None으로 초기화 되긴 하지만, 가끔 꼬이는 것 방지
if 'link' in bakery_data.columns: 
    del bakery_data['link']

# link 컬럼 추가
bakery_data['link'] = None

# 제과점 데이터와 매출 링크 데이터 연결
for i in range(len(bakery_data)):  # 셀 하단 참고 1 참조
    for j in range(len(linked_df)):
        if bakery_data.loc[i, '상권_코드_명'] == linked_df.iloc[j, 0]:  # 상권코드명이 동일할 때 
            bakery_data.loc[i, 'link'] = linked_df.iloc[j, 1]  # link 컬럼에 링크 번호 입력

# 예상 구동시간 1분 10초

# 참고 1) 
# 현재 구동 방식이 제과점 데이터의 길이를 range하여 순차적인 인덱스를 사용하기 때문에, 위에서 reset_index를 사용함
# len(bakery_data)를 사용하는 것이 아니라 bakery_data.index 를 사용하면 초기화하지 않아도 됨

In [ ]:
# 분기별로 매출 데이터 나누기
# 분기단위 유동인구 데이터를 사용하기 때문에, 매출데이터도 분기단위로 구분
# 이 셀 부터 분기로 나눈 유동인구 데이터를 기준으로 사용함

# 만약 유동인구 데이터가 분기 구분 없이 통합되어 있다면, 이 셀은 시행하지 않아도 됨

# 분기별 저장할 데이터 프레임 생성
bd_qt1 = bd_qt2 = bd_qt3 = bd_qt4 = pd.DataFrame()

# 분기별 데이터 저장 시행
for i in range(len(bakery_data)):
    target_row = pd.DataFrame(bakery_data.iloc[i]).T
    target_qt = bakery_data.loc[i, '기준_분기_코드']
    for j in range(4):  # 4분기 까지이므로 4
        if target_qt == j+1:  # range는 0,1,2,3 을 나타내고, 변수명에는 1,2,3,4 이므로 +1을 해줌
            globals()["bd_qt"+str(j+1)] = pd.concat([globals()["bd_qt"+str(j+1)], target_row], axis=0, ignore_index=True)
            # globals()[변수명]은 반복문 안에서 변수를 동적으로 사용하고 싶을 때 사용하는 구문
            # list나 tuple, dict를 사용하여 변수 배열을 하나 만들고, 순차적으로 불러오는 식으로 변수를 사용해도 동작은 비슷하다.

In [ ]:
# NaN 포함 행 제거하기
# 기존 매출데이터에는 NaN이 존재하지 않는데, link를 추가하면서 NaN이 생성되었다.
# 이 NaN은 link에만 존재하는 결손치로, 이는 상권_코드_명 컬럼에 '역'이 포함되지 않는 데이터를 의미한다.
# 이를 제거함으로써, '역' 이라는 데이터만 남긴다.

dna_bd_qt1 = bd_qt1.dropna(axis=0, inplace=False)

In [ ]:
# link를 사용하여 매출 합산
# 상권_코드_명에 같은 역이지만 여러가지 이름으로 기재되어 있기에, 이를 링크 데이터를 사용하여 하나로 묶는다.
# 링크 데이터는 역명이 같으면 링크 데이터 번호도 같다.
# 동대문역 = 동대문역 1번 = 동대문역 3번

# 합산 횟수 확인용 컬럼 데이터 추가
dna_bd_qt1['div_num'] = 1

# 합산
group_sales = dna_bd_qt1.groupby(['link']).agg('sum')
group_sales = group_sales.reset_index()  # groupby를 사용하면 링크가 인덱스화 되는데, 이 링크는 추후 merge에 사용되므로 추출해놓기

In [ ]:
# 정리 - groupby.agg('sum')을 통하여 잘못 합산된 수치나 문자열 정리

# 원본 보존
group_sales_cleanse = group_sales.copy()

# 역명 추가 및 기존 상권_코드_명 삭제
# sum을 하면서 상권_코드_명이 합쳐졌는데, 길이가 일정치 않아 정리하기 어려우므로 지하철 역명을 대체 사용
group_sales_cleanse = pd.merge(group_sales_cleanse, linked_stt_df, how='outer', on='link')
group_sales_cleanse.drop(columns=['상권_코드_명'], axis=1, inplace=True)

# 정리
# merge를 시행하면 int가 float로 되는 경우가 많다. 원본 매출 데이터내에서 float를 사용하지 않으므로 int로 변경
target_float_arr = []

for i in group_sales_cleanse.columns:
    if group_sales_cleanse[i].dtype == (r"float"):  # dtype은 str로 나오고, 이 결과가 float면 배열에 추가
        target_float_arr.append(i)

for i in target_float_arr:  # float인 배열을 int로 astype을 사용해서 변경
    group_sales_cleanse[i] = group_sales_cleanse[i].astype('int64')

# 이상 합산 수치 및 문자열 정리
miss_sum_arr_num = ['기준_분기_코드','기준_년_코드','상권_코드']  # 수치가 잘못 합산된 컬럼명
miss_sum_arr_str = ['상권_구분_코드_명','서비스_업종_코드_명','서비스_업종_코드','상권_구분_코드','자치구','동']  # 문자가 잘못 합산된 컬럼명

for i in range(len(group_sales_cleanse)):  # 모든 데이터에 대해서 시행
    total_sum = group_sales_cleanse.loc[i, 'div_num']  # 몇번 합쳐졌는지 확인
    
    for j in miss_sum_arr_num:  # 수치형 컬럼 정리
        group_sales_cleanse.loc[i, j] = int(group_sales_cleanse.loc[i, j] / total_sum)

    for j in miss_sum_arr_str:  # 문자형 컬럼 정리
        group_sales_cleanse.loc[i, j] = group_sales_cleanse.loc[i, j][0:(int(len(group_sales_cleanse.loc[i, j])/total_sum))]

## 유동인구 데이터 추출 및 정리

In [ ]:
# 2021_1분기 유동인구 데이터는 qt1_population_data
# 제과점 데이터와 동일하게, 링크를 추가한다.

if 'link' in qt1_population_data.columns:
    del qt1_population_data['link']

qt1_population_data['link'] = None

for i in range(len(qt1_population_data)):
    for j in range(len(linked_stt_df)):
        if qt1_population_data.loc[i, '역명'] == linked_stt_df.iloc[j, 0]:
            qt1_population_data.loc[i, 'link'] = linked_stt_df.iloc[j, 1]

In [ ]:
# 유동인구 데이터, 요일-시간대별 컬럼 분리
# 기존 유동인구 데이터가 요일별로 행이 나누어져 있기 때문에, 이를 하나의 역명 행으로 합쳐주는 작업
# 자세한 데이터 상태는 csvdata/2021_1qt.csv 참고

# 유동인구 데이터를 정제하는 과정이 달라지면 이 단계는 변할 수 있다.

# 원본 보존
qt1_population_data_cleanse = qt1_population_data.copy()

# 컬럼명 및 요일명으로 사용하고 있는 데이터 정리
timezone = ['00시11시', '11시17시', '17시24시', '총합', '평균']
weekdays_name =['월','화','수','목','금','토','일']
col_names_arr = []

# 신규 컬럼명 생성
for i in weekdays_name:
    for j in timezone:
        new_col_name = i + "_" + j  # 요일_시간대 의 문자열로 결합
        col_names_arr.append(new_col_name)  # 신규 컬럼명 배열 생성

# 신규 컬럼 추가 및 초기화
for i in col_names_arr:
    qt1_population_data_cleanse[i] = 0

# 해당하는 컬럼에 데이터 입력
for i in range(len(qt1_population_data_cleanse)):
    wd_name = qt1_population_data_cleanse.loc[i, '요일']  # 해당 요일 확인
    for j in col_names_arr:
        if wd_name==j[0]:  # j는 신규 컬럼명으로 '월_00시11시'의 형태를 가진다. j[0]은 '월_00시11시'에서 idx 0인 월, 요일을 가르킨다.
            qt1_population_data_cleanse.loc[i, j] = qt1_population_data_cleanse.loc[i, j[2:]] 
            # j[2:]는 idx 2 이후인 00시11시를 가르킨다. 기존 컬럼명인 00시11시의 데이터를 신규 요일_시간대 컬럼에 가져오는 것

In [ ]:

# 분리된 유동인구 데이터 하나의 행으로 합치기
# 윗단계 셀로 나온 결과는, 각 요일의 시간대 결과와, 분리되어 입력된 결과가 동시에 입력되어 있다.
''' 데이터 구성 예시
    역명    요일 00시11시 ... 17시24시 ... 월_00시11시 ... 일_17시24시 ...
가락시장      월      100          147             100              0
...
가락시장      일      160           87               0             87
...
'''
# 기존 데이터를 제거함과 동시에 나누어져 있는 요일_시간대 컬럼을 합치는 과정

# 유동인구 데이터를 정제하는 과정이 달라지면 이 단계는 변할 수 있다.

# 역명과 링크를 기준으로 합치기
qt1_population_simple_row = qt1_population_data_cleanse.groupby(['역명', 'link']).agg('sum')

# 기존 유동인구 데이터 제거, 컬럼단위 제거
qt1_population_simple_row.drop(['00시11시','11시17시','17시24시','총합', '평균'], axis=1, inplace=True)

# 역명과 링크는 이후 병합에 사용되므로, 추출한다.
qt1_population_simple_row.reset_index(drop=False, inplace=True)

## 데이터 병합하여 하나의 데이터셋으로 만들기

In [ ]:
# 제과점 데이터와 유동인구 합산 데이터 병합
qt1_data = pd.merge(group_sales_cleanse, qt1_population_simple_row, how='outer',on=['link', '역명'])

qt1_data.dropna(axis=0, inplace=True)

# qt1_data.to_csv("./qt1_money_popular_data.csv", encoding='utf-8-sig')
